In [1]:
import json
import sys, re

import requests

from copy import deepcopy
from pprint import pprint

In [2]:
"""someone's function generating visuals"""   
def getsimsongs(artist, song):   # gives dict of sim songs
    urlraw = f'https://ws.audioscrobbler.com/2.0/?method=track.getsimilar&artist={artist}&track={song}&api_key=70894b101e887cf7fd9443bd5067f675&format=json'

    dicy = {}

    r = requests.get(urlraw)
    tmp = r.json()
#     pprint(tmp)

    for i in range(len(tmp['similartracks']['track'])):
        songname = tmp['similartracks']['track'][i]['name']
        artistname = tmp['similartracks']['track'][i]['artist']['name']
#         print(f'{artistname}|{songname}')

        dicy[f'{artistname}|{songname}'] = 1

    return dicy
    
    
def getsimartists(artist):
    urlraw = f'http://ws.audioscrobbler.com/2.0/?method=artist.getsimilar&artist={artist}&api_key=70894b101e887cf7fd9443bd5067f675&format=json'

    dicy = {}

    r = requests.get(urlraw)
    tmp = r.json()
#     pprint(tmp)

    for i in range(len(tmp['similarartists']['artist'])):
        artistname = tmp['similarartists']['artist'][i]['name']
#         print(artistname)

        dicy[f'{artistname}'] = 1

    return dicy
    


def match(dicy0, dicy1):   # gives matching dict of two dicts
    pot = {}
    
    for k0 in dicy0:
        for k1 in dicy1:
            if k0 == k1:
                pot[k1] = 1

    return pot


def addmore(dicy, n):    # adds new sim songs by specified pair
    pair = list(dicy)[n]
#     print(pair)
    
    if '|' in pair:
        artistpair = pair.split('|')[0]
        songpair = pair.split('|')[1]
        
        try:
            dicypair = getsimsongs(artistpair, songpair)
#             print(dicypair)
        
            dicynew = {**dicy, **dicypair}
        except:
            dicynew = dicy
        
#             print(f'FAILED searching... {pair}')
    else:
        artistpair = pair
        
        try:
            dicypair = getsimartists(artistpair)
#             print(dicypair)
        
            dicynew = {**dicy, **dicypair}
        except:
            dicynew = dicy

    
    return dicynew, pair


def iteration(dicy0, dicy1, hist0={}, hist1={}):  
    ##########################################################
    # check if sets inital match
    ##########################################################
#     print('data0', len(dicy0))
#     print('data1', len(dicy1))

    res = match(dicy0, dicy1)

    if len(res) >= 1:
#         print(len(res), res)
#         print(f'stop after add 0 pair(s)')
        pass
    ##########################################################
    # expand song by song until match
    ##########################################################
    else:
        for i in range(max(len(dicy0), len(dicy1))):
            ind0 = 'n'
            ind1 = 'n'

            try:
                test0 = list(dicy0)[i]
            except:
                test0 = ''

            try:
                test1 = list(dicy1)[i]
            except:
                test1 = ''

            if test0.strip() and test0 not in hist0:
                dicy0, pair0 = addmore(dicy0, i)

#                 print('dicy0',len(dicy0))

                hist0[pair0] = 1

                ind0 = 'y'

            if test1.strip() and test1 not in hist1:
                dicy1, pair1 = addmore(dicy1, i)

#                 print('dicy1', len(dicy1))

                hist1[pair1] = 1

                ind1 = 'y'

            if not (ind0 == 'n' and ind1 == 'n'):
                res = match(dicy0, dicy1)

                if len(res) >= 1:
#                     print(len(res), res)
#                     print(f'stop after add {i + 1} pair(s)')
                    break

    return res, dicy0, dicy1, hist0, hist1

In [3]:
def mainsongs(artist0, song0, artist1, song1):
    try:
        d0 = getsimsongs(artist0, song0)
    except:
        print(f"""your artist ({artist0}) or song ({song0}) is WRONG""")
        
    try:
        d1 = getsimsongs(artist1, song1)
    except:
        print(f"""your artist ({artist1}) or song ({song1}) is WRONG""")
        
    # first iteration pair by pair
    try:
        res, d00, d11, h00, h11 = iteration(d0, d1)
        
        print(f'SONG(S) YOU MAY LIKE : {len(res)}')

        for k in res:
            print(f"{k.split('|')[0]} - {k.split('|')[1]}")
    except:
        pass


def mainartists(artist0, artist1):
    try:
        d0 = getsimartists(artist0)
    except:
        print(f"""your artist ({artist0}) is WRONG""")
    
    try:
        d1 = getsimartists(artist1)
    except:
        print(f"""your artist ({artist1}) is WRONG""")
        
    # first iteration pair by pair
    try:
        res, d00, d11, h00, h11 = iteration(d0, d1)

        print(f'ARTIST(S) YOU MAY LIKE: {len(res)}')

        for k in res:
            print(k)
    except:
        pass

In [4]:
"""let user pick any number of artists and respective songs"""
# artist0 = "eminem"
# song0 = "mockingbird"

# artist0 = "nickelback"
# song0 = "far away"

# artist0 = "justin bieber"
# song0 = "baby"

artist0 = 'the chainsmokers'
song0 = 'something just like this'

# artist0 = input('pick artist? ').title()
# song0 = input(f'pick song for {artist0}? ')


print()
# artist1 = "Nicki Minaj"
# song1 = "anaconda"

# artist1 = "drake"
# song1 = "god's plan"

# artist1 = 'taylor swift'
# song1 = 'look what you made me do'

artist1 = 'lady gaga'
song1 = 'just dance'

# artist1 = input('pick artist? ').title()
# song1 = input(f'pick song for {artist1}? ')

In [5]:
mainsongs(artist0, song0, artist1, song1)
print()
mainartists(artist0, artist1)

SONG(S) YOU MAY LIKE : 13
Clean Bandit - Rockabye (feat. Sean Paul & Anne-Marie)
Selena Gomez - Wolves
Calvin Harris - How Deep is your love
Dua Lipa - New Rules
Black Eyed Peas - I Gotta Feeling
Ellie Goulding - Burn
Zara Larsson - Ain't My Fault
Ke$ha - Die Young
Clean Bandit - Tick Tock (feat. 24kGoldn)
Ke$ha - TiK ToK
Selena Gomez - Come & Get It
Troye Sivan - My My My!
Alexandra Stan - Mr. Saxobeat

ARTIST(S) YOU MAY LIKE: 7
Clean Bandit
Calvin Harris
David Guetta
Zara Larsson
Bebe Rexha
Rita Ora
Halsey
